<a href="https://colab.research.google.com/github/rahul-kumar-yadav/Story_Point_keras_Embedding/blob/main_/Story_point_estimation_Keras_Embedding_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# load dataset
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
import string
# from word2vec_keras import Word2VecKeras
# Word2vec
import gensim
from tensorflow.keras import layers


# Scikit learn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Keras
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Embedding, LSTM
from keras import utils
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Utility
import numpy as np
import tarfile
import pickle
import tempfile
import os
import time
import logging
import multiprocessing

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)




df = pd.read_csv("jirasoftware.csv", usecols=['issuekey', 'title', 'description', 'storypoint'])
# df.isnull().sum()
	
df = df.dropna(how='any')

df.head()

df.loc[df.storypoint <= 3, 'storypoint'] = 0 #small
df.loc[(df.storypoint > 3) & (df.storypoint <= 8), 'storypoint'] = 1 #medium
df.loc[df.storypoint > 8, 'storypoint'] = 2 #big
htmltokens = ['{html}','<div>','<pre>','<p>', '</div>','</pre>','</p>']

def removeHtml(text):
  for w in htmltokens:
        text = text.replace(w, '')
  return text
df['description'] = df["description"].apply(lambda x: re.sub(r'http\S+','',x))
def cleanData(text):
  result = ''
  str1=""
  list1=[]
  for c in text:
    if c not in string.punctuation:
      list1.append(c)
      str1 = "".join(list1)
    else:
      list1.append(" ")
      str1 = "".join(list1)
  
  char_string = ' '.join([w for w in str1.split() if len(w) > 2])


  text_words = char_string.split()
  resultwords=[word for word in text_words if word not in stopwords.words('english')]

  if len(resultwords) > 0:
    result=' '.join(resultwords)
  else:
    print("empty text")
  return result


df['label_title_desc'] = df['title'].str.lower() + ' - ' + df['description'].str.lower()
df['label_title_desc'] = df['label_title_desc'].apply(lambda x:removeHtml(str(x)))
df['title_desc'] = df['label_title_desc'].apply(lambda x: cleanData(str(x)))

X_train_split, X_test, y_train_split, y_test = train_test_split(df['title_desc'], df['storypoint'], train_size=0.8, random_state=42)
from collections import Counter

def SimpleOverSample(_xtrain, _ytrain):
    xtrain = list(_xtrain)
    ytrain = list(_ytrain)

    samples_counter = Counter(ytrain)
    max_samples = sorted(samples_counter.values(), reverse=True)[0]
    for sc in samples_counter:
        init_samples = samples_counter[sc]
        samples_to_add = max_samples - init_samples
        if samples_to_add > 0:
            #collect indices to oversample for the current class
            index = list()
            for i in range(len(ytrain)):
                if(ytrain[i] == sc):
                    index.append(i)
            #select samples to copy for the current class    
            copy_from = [xtrain[i] for i in index]
            index_copy = 0
            for i in range(samples_to_add):
                xtrain.append(copy_from[index_copy % len(copy_from)])
                ytrain.append(sc)
                index_copy += 1
    return xtrain, ytrain

X_train,y_train = SimpleOverSample(X_train_split, y_train_split)

vocab_size = 20000
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
num_classes = len(label_encoder.classes_)
y_train = utils.to_categorical(y_train, num_classes)

y_test = label_encoder.fit_transform(y_test)
num_classes = len(label_encoder.classes_)
y_test = utils.to_categorical(y_test, num_classes)

X_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_train]
X_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_test]

max_length = 100
X_train = pad_sequences(X_train, maxlen=max_length, padding='pre')
X_test = pad_sequences(X_test, maxlen=max_length, padding='pre')

model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_length))
# model.add(Flatten())
model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.2))
model.add(Dense(128, activation='relu'))
# layers.Dropout(0.3),
# layers.BatchNormalization(),
model.add(Dense(64, activation='relu'))
# layers.Dropout(0.3),
# layers.BatchNormalization(),
model.add(Dense(32, activation='relu'))
# layers.Dropout(0.3),
# layers.BatchNormalization(),
model.add(Dense(16, activation='relu'))
# layers.Dropout(0.3),
# layers.BatchNormalization(),
model.add(Dense(num_classes, activation='softmax'))
early_stopping = EarlyStopping(patience=30, min_delta=0.001, restore_best_weights=True)
callbacks = [early_stopping]

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=60, callbacks=callbacks, verbose=1)



loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
print('Training Accuracy is {}'.format(accuracy*100))



loss, accuracy = model.evaluate(X_test,y_test)
print('Testing Accuracy is {} '.format(accuracy*100))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_40 (Embedding)     (None, 100, 32)           640000    
_________________________________________________________________
lstm_37 (LSTM)               (None, 128)               82432     
_________________________________________________________________
dense_179 (Dense)            (None, 128)               16512     
_________________________________________________________________
dense_180 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_181 (Dense)            (None, 32)                2080      
_________________